In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit_ibm_runtime.program import UserMessenger
from qiskit_ibm_provider import IBMProvider

from openqaoa.problems import MaximumCut
import oq_runtime_qaoa as oq_run
import matplotlib.pyplot as plt

In [ ]:
service = QiskitRuntimeService(instance="<YOUR IBMQ ACCESS INSTANCE>")

In [ ]:
#prints the credentials associated with an active IBMQ account
service.active_account()

In [ ]:
user_messenger = UserMessenger()

maxcut = MaximumCut.random_instance(n_nodes=15, edge_probability=0.5)
maxcut_qubo = maxcut.qubo
maxcut_qubo_dict = maxcut_qubo.asdict()

In [ ]:
from openqaoa.utilities import plot_graph

In [ ]:
plot_graph(maxcut.G)

In [ ]:
optimizer_dict = dict({"maxiter":150, "method":"COBYLA", "optimization_progress":True})
program_inputs = {
    "qubo_dict":maxcut_qubo_dict,
    "p":1,
    "n_shots":5000,
    "circuit_optimization_level":2,
    "optimizer_dict":optimizer_dict
}

In [ ]:
backend="ibmq_guadalupe"

In [ ]:
with Session(service=service, backend=backend) as session:
    
    # Submit a request to the OpenQAOA runtime within the session.
    runtime_job = service.run(program_id = 'PROGRAM_ID_CUSTOM_RUNTIME',
                              inputs = program_inputs,
                              options = {"backend": backend})
    
    print(f"QAOA results: {runtime_job.result()}")

    # Close the session only if all jobs are finished
    # and you don't need to run more in the session.
    session.close()

In [ ]:
plt.plot(runtime_job.result()['intermediate']['cost'])
plt.xlabel("iterations")
plt.ylabel("Cost")
plt.title("15 qubit QAOA optimization on ibmq_guadalupe")

In [ ]:
from openqaoa import create_device, QAOA

In [ ]:
local_qasm_simulator = create_device("local", "qiskit.qasm_simulator")

In [ ]:
q = QAOA()
q.set_device(local_qasm_simulator)
q.set_backend_properties(n_shots=5000)
q.set_classical_optimizer(method="COBYLA", optimization_progress=True,maxiter=150)

In [ ]:
q.compile(maxcut_qubo)

In [ ]:
q.optimize()

In [ ]:
q.result.plot_cost(label="15 qubit QAOA on qasm_simulator")